## [Νικόλαος Μανιάτης](mailto:nikolaosmaniatis@mail.ntua.gr)
## Α.Μ.: 03400097

<h2><b><i>Radial Basis Function Neural Network</i></b></h2>

Σκοπός της παρούσας άσκησης είναι η κατηγοριοποίηση χειρόγραφων ψηφίων από το [MNIST dataset](https://www.wikiwand.com/en/MNIST_database) με τη χρήση νευρωνικών δικτύων [RBF](https://en.wikipedia.org/wiki/Radial_basis_function) (RBF). Για την κατηγοροποίηση των χειρόγραφων ψηφίων, χρησιμοποιείται ο αλγόριθμος K-means. Στη κλασική υλοποίηση του K-means όπου έχουμε το κέντρο του cluster και ταξινομομούμε κάθε σημείο με βάση την απόσταση από τα κέντρα. Σε αυτή την περίπτωση μπορεί να υπάρξουν σημεία που να μην μπορούν να αντιστοιχηθούν σε κάποιο cluster. Με τη χρήση του νευρωνικού δικτύου RBF κάθε σημείο ταξινομείται σε ομάδες με συγκεκριμένο κέντρο, γνωρίζοντας όμως τις αποστάσεις μεταξύ των κέντρων αλλά και με διαστήματα εμπιστοσύνης ως προς την πρόβλεψή μας. Με αυτό το τρόπο όλα τα σημεία αντιστοιχίζονται σε ένα cluster. Για να είναι ομαλή η ταξινόμηση των στοιχείων στα clusters μπορεί να χρησιμοποιηθεί μια εκθετική συνάρτηση υψωμένη στην αρνητική τιμή της απόστασης ως γινόμενο με έναν βαθμωτό συντελεστή beta, όπως φαίνεται παρακάτω:

![alt text](https://miro.medium.com/max/145/1*MIay3aIlpT18yewOfnvTiQ.png)

Στα πλαίσια του παραδείγματος για το β χρησιμοποιήθηκε ο παρακάτω τύπος

![alt text](https://miro.medium.com/max/608/1*YEcI_P6orY917fQrzHQEjQ.png)

Όπου k ο αριθμός των clusters και Dmax η μέγιστη ευκλείδια απόσταση μεταξύ των κέντρων. Το dataset που θα χρησιμοποιήσετε είναι διαθέσιμο [εδώ](https://drive.google.com/file/d/1WrVPB3C1vIApWNz5BaSuNmHFopf5htCg/view?usp=sharing) (θα πρέπει να το φορτώσετε στο Colab για να το τρέξετε). 

*Οι ερωτήσεις της άσκησης είναι μαζί με το αντίστοιχο βήμα στον κώδικα*


**Βήμα 1ο**

Τρέχετε τον κώδικα στον οποίο και ορίζονται οι συναρτήσεις που θα μας χρειαστούν. Στα πλαίσια της άσκησης χρησιμοποιούμε μία παραμετροποιημένη έκδοση της συνάρτησης K-Means, όπου μας επιστρέφει τα κέντρα των ομάδων (clusters) καθώς και την τυπική απόκλιση τους.

In [ ]:
"""''

Copyright © 2020 Tarlan Ahadli

Permission is hereby granted, free of charge, to any person obtaining a
copy of this software and associated documentation files (the “Software”),
to deal in the Software without restriction, including without limitation
the rights to use, copy, modify, merge, publish, distribute, sublicense,
and/or sell copies of the Software, and to permit persons to whom the Software
is furnished to do so, subject to the following conditions:

The above copyright notice and this permission notice shall be included
in all copies or substantial portions of the Software.

THE SOFTWARE IS PROVIDED “AS IS”, WITHOUT WARRANTY OF ANY
KIND, EXPRESS OR IMPLIED, INCLUDING BUT NOT LIMITED TO THE
WARRANTIES OF MERCHANTABILITY, FITNESS FOR A PARTICULAR PURPOSE
AND NONINFRINGEMENT. IN NO EVENT SHALL THE AUTHORS OR COPYRIGHT
HOLDERS BE LIABLE FOR ANY CLAIM, DAMAGES OR OTHER LIABILITY,
WHETHER IN AN ACTION OF CONTRACT, TORT OR OTHERWISE, ARISING
FROM, OUT OF OR IN CONNECTION WITH THE SOFTWARE OR THE USE OR
OTHER DEALINGS IN THE SOFTWARE.

''"""

import numpy as np


def get_distance(x1, x2):
    sum = 0
    for i in range(len(x1)):
        sum += (x1[i] - x2[i]) ** 2
    return np.sqrt(sum)


def kmeans(X, k, max_iters):
    centroids = X[np.random.choice(range(len(X)), k, replace=False)]
    # centroids = [np.random.uniform(size=len(X[0])) for i in range(k)]

    converged = False
    current_iter = 0

    while (not converged) and (current_iter < max_iters):

        cluster_list = [[] for i in range(len(centroids))]

        for x in X:  # Go through each data point
            distances_list = []
            for c in centroids:
                distances_list.append(get_distance(c, x))
            cluster_list[int(np.argmin(distances_list))].append(x)

        cluster_list = list((filter(None, cluster_list)))

        prev_centroids = centroids.copy()

        centroids = []

        for j in range(len(cluster_list)):
            centroids.append(np.mean(cluster_list[j], axis=0))

        pattern = np.abs(np.sum(prev_centroids) - np.sum(centroids))

        print('K-MEANS: ', int(pattern))

        converged = (pattern == 0)

        current_iter += 1

    return np.array(centroids), [np.std(x) for x in cluster_list]


CPU times: user 12 µs, sys: 0 ns, total: 12 µs
Wall time: 14.8 µs


**Βήμα 2ο**

Τρέχουμε την κλάση για το RBFNN

---

***Ερώτηση 1***

Να αναφέρετε επιγραμματικά με ποιον τρόπο λειτουργεί ένα νευρωνικό δίκτυο RBF.  
***Ερώτηση 2***

Στα πλαίσια του παραδείγματος, για κάθε αριθμό (κλάση) έχουμε μία μόνο ομάδα (cluster). Τι θα συνέβαινε αν είχαμε παραπάνω από μία ομάδες που θα αντιστοιχούσαν σε μία κλάση; Πώς λύνεται αυτό μέσω του RBF;


- **Ερώτηση 1**:  
Ένα Radial Basis Function Network αποτελείται από 3 layers, το input, ένα hidden layer με μια μη γραμμική RBF activation function $h(x)$, και ένα output layer. Συγκεκριμένα, αν έχουμε ένα RBF network με $Ν$ κρυμμένους νευρώνες, ως input ένα διάνυσμα $x = (x_1,x_2,\dots,x_d)^\top$ μήκους $d$, και θέλουμε να υπολογίσουμε το output μήκους $p$, $y_i$ για $i=1,2,\dots,p$, όπου $w_{ij}$ είναι το βάρος της σύνδεσης από τον κρυμμένο νευρώνα j
προς το νευρώνα εξόδου i και $w_{i0}$ είναι το bias του νευρώνα εξόδου i: 
$$y_i(x) = w_{i0} + \displaystyle\sum_{i=0}^N w_{ij}h_j(x)$$
Όπως αναφέραμε, $h(x)$ είναι η συνάρτηση πυρήνα, όπου συνήθως σε αυτές υπάρχει κάποιο σημείο (που ονομάζεται κέντρο) για το οποίο παρέχουν μέγιστη τιμή, και καθώς απομακρυνόμαστε ακτινικά από το κέντρο η τιμή της συνάρτησης μειώνεται και σχεδόν μηδενίζεται για σημεία που είναι πολύ μακριά από το κέντρο.
Τυπικό παράδειγμα μιας τέτοιας συνάρτησης είναι ο Gaussian Kernel:
$$ h(x) = \exp(-\frac{\parallel x-w_j\parallel^2}{2\sigma_j^2}$$

- **Ερώτηση 2**:  
Αν είχαμε παραπάνω από ένα cluster να αντιστοιχεί σε κάθε κλάση, τότε θα μπορούσαμε να πάρουμε τον γραμμικό συνδιασμό αυτών των κλάσεων, επομένως το τελικό output που θα έβγαινε από το δίκτυο θα εξαρτάται από όλα τα RBFs. Η δυσκολία σε αυτήν την περίπτωση είναι να βρούμε τα κατάλληλα βάρη $w_{ij}$ που προσεγγίζει καλύτερα την γραμμική σχέση των πολλαπλών clusters για μια κλάση. Στα RBFs αυτό που μπορούμε να κάνουμε για να το λύσουμε αυτό είναι να χρησιμοποιήσουμε την μέθοδο της γραμμικής παλινδρόμησης με χρήση ελαχίστων τετραγώνων.





In [ ]:
class RBF:

    def __init__(self, X, y, tX, ty, num_of_classes,
                 k, std_from_clusters=True):
        self.X = X
        self.y = y

        self.tX = tX
        self.ty = ty

        self.number_of_classes = num_of_classes
        self.k = k
        self.std_from_clusters = std_from_clusters

    def convert_to_one_hot(self, x, num_of_classes):
        arr = np.zeros((len(x), num_of_classes))
        for i in range(len(x)):
            c = int(x[i])
            arr[i][c] = 1
        return arr

    def get_rbf(self, x, c, s):
        distance = get_distance(x, c)
        return 1 / np.exp(-distance / s ** 2)

    def get_rbf_as_list(self, X, centroids, std_list):
        RBF_list = []
        for x in X:
            RBF_list.append([self.get_rbf(x, c, s) for (c, s) in zip(centroids, std_list)])
        return np.array(RBF_list)

    def fit(self):

      self.centroids, self.std_list = kmeans(self.X, self.k, 1000)

      if not self.std_from_clusters:
          dMax = np.max([get_distance(c1, c2) for c1 in self.centroids for c2 in self.centroids])
          self.std_list = np.repeat(dMax / np.sqrt(2 * self.k), self.k)

      RBF_X = self.get_rbf_as_list(self.X, self.centroids, self.std_list)

      self.w = np.linalg.pinv(RBF_X.T @ RBF_X) @ RBF_X.T @ self.convert_to_one_hot(self.y, self.number_of_classes)

      RBF_list_tst = self.get_rbf_as_list(self.tX, self.centroids, self.std_list)

      self.pred_ty = RBF_list_tst @ self.w

      self.pred_ty = np.array([np.argmax(x) for x in self.pred_ty])

      diff = self.pred_ty - self.ty

      print('Accuracy: ', len(np.where(diff == 0)[0]) / len(diff))

**Βήμα 3ο**

Τρέχουμε τη συνάρτηση όπου το RBFNN γίνεται fit. Αρχικά τρέχει ο αλγόριθμος k-means για να βρούμε τα κέντρα και την τυπική απόκλιση των clusters. Στη συνέχεια, μπορούμε να χρησιμοποιήσουμε το ίδιο beta για όλα τα clusters


---
***Ερώτηση 3***

Με ποιούς τρόπους μπορούμε να υπολογίσουμε τα βάρη στα hidden layers; Ποιες διαφορές υπάρχουν στους τρόπους αυτούς; Ποιος παρέχει τη καλύτερη δυνατή λύση;


**Ερώτηση 3**:   
Υπάρχουν διάφοροι τρόποι υπολογισμού των βαρών στα hidden layers, ή αλλιώς να εκπαιδεύσουμε το δίκτυο.
 - **Ενιαία εκπαίδευση δικτύου**: Χρήση Gradient Descent ή Batch Gradient Descent προς ελαχιστοποίηση της τετραγωνικής συνάρτησης σφάλματος. Συνήθως προτιμάται η Batch Gradient Descent. Εδώ είναι σημαντική η αρχικοποίηση των κέντρων και ακτίνων των RBFs καθώς μπορεί ο gradient descent να παγιδευτεί σε τοπικά ελάχιστα. Συνήθως προτιμάται η αρχικοποίηση των κεντρών των νευρώνων των RBFs να είναι το κέντρο κάποιου cluster των δεδομένων.

 - **Εκπαίδευση δύο σταδίων**: Εδώ στο **πρώτο στάδιο** χρησιμοποιείται το σύνολο των δεδομένων εκπαίδευσης για τον καθορισμό νευρώνων RBF, και αγνοείται η πληροφορία για το label της κλάσης. Στη συνέχεια εφαρμόζεται κάποια μέθοδος clustering π.χ. k-means ώστε να προκύψουν τα κέντρα των ομάδων, και οι ακτίνες μέσω υπολογισμού της διασποράς των δεδομένων κάθε cluster. 
 Στο **δεύτερο στάδιο** έχουμε δύο επιλογές, η πρώτη είναι τα κέντρα και οι ακτίνες να παραμείνουν σταθερά, και μένει να βρεθούν τα weights και biases μόνο τυτου output layer. Στη δεύτερη περίπτωση, εφαρμόζουμε gradient descent όπως ακριβώς στην ενιαία εκπαίδευση, όμως η αρχικοποίηση των κέντρων και των ακτίνων γίνεται με τις τιμές που έχουν προκύψει από το πρώτο στάδιο. 

 Η καλύτερη δυνατή λύση γίνεται με την εκπαίδευση δύο σταδίων, όπου χρησιμοποιείται κάποιος clustering αλγόριθμος προκειμένου να γίνει η αρχικοποίηση στα κέντρα και τις ακτίνες των ομάδων, και στην πορεία η εκπαίδευση να γίνει με batch gradient descent. 


**Βήμα 4ο**

Σπάμε το dataset του MNIST σε training και testing και αφήνουμε το RBFNN να βγάλει το τελικό αποτέλεσμα.

---

***Ερώτηση 4***

Επιχειρήστε να αλλάξετε την τιμή του k σε 500 και στη συνέχεια σε 1000. Τι παρατηρείτε ως προς την εκτέλεση;

Τα αποτελέσματα είναι:
- k = 10: Accuracy:  63.7%, Χρόνος εκτέλεσης: 12 min 18 sec
- k = 500: Accuracy: 94.3%, Χρόνος εκτέλεσης: 2 hrs 19 min 44 sec
- k = 1000: Accuracy: 95.3%, Χρόνος Εκτέλεσης: 3 hrs 24 min 32 sec

Παρατηρούμε ότι για $k=10\rightarrow k=500$ έχουμε μεγάλη αύξηση στην ακρίβεια όμως σε βάρος μεγάλου υπολογιστικού κόστους. Για $k=500\rightarrow k=1000$ πάλι έχουμε μεγάλο υπολογιστικό κόστος για πολύ μικρή αύξηση στην ακρίβεια, επομένως κάτι τέτοιο δεν θα ήταν χρήσιμο. Είναι σημαντικό να σημειωθεί ότι μεγάλη αύξηση στους clusters είναι επικίνδυνη καθώς μπορεί να εμφανιστεί overfitting. 

In [ ]:
##################################
#LOAD DATA
import os 
from google.colab import drive
drive.mount('/content/drive')
data = np.load(os.getcwd()+'/drive/MyDrive/mnist_data.npy').astype(float)

train_y = data[0:2500, 0]
train_x = data[0:2500, 1:]

# by default the code was like this:
# test_y = data[0:300, 0]
# test_x = data[0:300, 1:]
#but it's wrong because we have the same samples in both train and test data (!!!)

test_y = data[2500:2800,0]
test_x = data[2500:2800,1:]


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


- k = 10

In [ ]:
%%time
RBF_CLASSIFIER = RBF(train_x, train_y, test_x, test_y, num_of_classes=10,
                     k=10, std_from_clusters=False)

RBF_CLASSIFIER.fit()

K-MEANS:  37174
K-MEANS:  5385
K-MEANS:  3947
K-MEANS:  2726
K-MEANS:  1987
K-MEANS:  1203
K-MEANS:  835
K-MEANS:  220
K-MEANS:  275
K-MEANS:  234
K-MEANS:  102
K-MEANS:  8
K-MEANS:  48
K-MEANS:  3
K-MEANS:  111
K-MEANS:  31
K-MEANS:  14
K-MEANS:  13
K-MEANS:  29
K-MEANS:  61
K-MEANS:  2
K-MEANS:  43
K-MEANS:  42
K-MEANS:  17
K-MEANS:  1
K-MEANS:  8
K-MEANS:  124
K-MEANS:  81
K-MEANS:  86
K-MEANS:  36
K-MEANS:  57
K-MEANS:  45
K-MEANS:  47
K-MEANS:  23
K-MEANS:  23
K-MEANS:  16
K-MEANS:  36
K-MEANS:  39
K-MEANS:  23
K-MEANS:  9
K-MEANS:  2
K-MEANS:  64
K-MEANS:  9
K-MEANS:  0
Accuracy:  0.6366666666666667
CPU times: user 12min 16s, sys: 1.47 s, total: 12min 17s
Wall time: 12min 18s


- k = 500

In [ ]:
%%time
RBF_CLASSIFIER = RBF(train_x, train_y, test_x, test_y, num_of_classes=10,
                     k=500, std_from_clusters=False)

RBF_CLASSIFIER.fit()

K-MEANS:  414320
K-MEANS:  415
K-MEANS:  18407
K-MEANS:  2248
K-MEANS:  2019
K-MEANS:  1169
K-MEANS:  8
K-MEANS:  686
K-MEANS:  0
Accuracy:  0.9433333333333334
CPU times: user 2h 19min 22s, sys: 15.3 s, total: 2h 19min 37s
Wall time: 2h 19min 44s


- k = 1000

In [ ]:
%%time
RBF_CLASSIFIER = RBF(train_x, train_y, test_x, test_y, num_of_classes=10,
                     k=1000, std_from_clusters=False)

RBF_CLASSIFIER.fit()

K-MEANS:  576550
K-MEANS:  18701
K-MEANS:  4126
K-MEANS:  6255
K-MEANS:  353
K-MEANS:  0
Accuracy:  0.9533333333333334
CPU times: user 3h 24min 1s, sys: 20.6 s, total: 3h 24min 22s
Wall time: 3h 24min 32s
